In [39]:
import openai, json
from pprint import pprint

client = openai.OpenAI()
messages = []

In [40]:
def get_weather(city):
    return "33도 입니다."

FUNCTION_MAP = {"get_weather": get_weather}

In [41]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather or a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of."
                    }
                },
                "required": ["city"]
            },
        },
    }
]

In [42]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    } 
                    for tool_call in message.tool_calls
                ],
            }
        )
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments - {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )
        call_ai()
    else:
        messages.append(
            {
                "role": "assistant",
                "content": message.content
            }
        )
        print(f"AI: {message.content}")



def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)
    message = response.choices[0].message.content



In [43]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({
            "role": "user",
            "content": message
        })
        print(f"User: {message}")
        call_ai()

        

User: my name is doun
AI: Nice to meet you, Doun! How can I assist you today?
User: what is my name?
AI: Your name is Doun.
User: what is the weather of seoul
Calling function: get_weather with {"city":"Seoul"}
Ran get_weather with args {'city': 'Seoul'} for a result of 33도 입니다.
AI: The weather in Seoul is 33 degrees Celsius.
User: 내가 어떤 도시의 온돌르 물어봤지?
AI: 당신은 서울의 날씨를 물어보셨습니다.
User: 좋은데?
AI: 감사합니다! 더 궁금한 것이 있으면 언제든지 물어보세요. 어떻게 도와드릴까요?
User: 벼ㅑㅅ
AI: 무슨 뜻인지 잘 이해하지 못했습니다. 다시 말씀해 주시겠어요?


In [46]:
messages

[{'role': 'user', 'content': 'my name is doun'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Doun! How can I assist you today?'},
 {'role': 'user', 'content': 'what is my name?'},
 {'role': 'assistant', 'content': 'Your name is Doun.'},
 {'role': 'user', 'content': 'what is the weather of seoul'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_goUp3jKEuq9QsBosN1sQEiOT',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_goUp3jKEuq9QsBosN1sQEiOT',
  'name': 'get_weather',
  'content': '33도 입니다.'},
 {'role': 'assistant',
  'content': 'The weather in Seoul is 33 degrees Celsius.'},
 {'role': 'user', 'content': '내가 어떤 도시의 온돌르 물어봤지?'},
 {'role': 'assistant', 'content': '당신은 서울의 날씨를 물어보셨습니다.'},
 {'role': 'user', 'content': '좋은데?'},
 {'role': 'assistant',
  'content': '감사합니다! 더 궁금한 것이 있으면 언제든지 물어보세요. 어떻게 도와드릴까요?'},
 {'role': 'user', 'content': '벼ㅑㅅ'},
 {'role': 'assist